<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Homework9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL=f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL=f"{PREFIX}/hair_classifier_v1.onnx"
!wget $DATA_URL
!wget $MODEL_URL

--2025-12-05 13:37:20--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-05T14%3A32%3A22Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-05T13%3A32%3A09Z&ske=2025-12-05T14%3A32%3A22Z&sks=b&skv=2018-11-09&sig=kp0CesACvxIYHUAdFYFqCnInClapK9tzi9epGSYIcHM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDk0MzY0MCwibmJmIjoxNzY0OTQxODQwLCJwYXR

In [9]:
pip install pillow

In [10]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [11]:
import tensorflow as tf
from tensorflow import keras

In [12]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 96.4 MB/s eta 0:00:00


In [13]:
import onnx

# Load your ONNX model
model = onnx.load("/content/hair_classifier_v1.onnx")

# Get the output names
output_names = [output.name for output in model.graph.output]

print("Output names:", output_names)


Output names: ['output']


In [14]:
!wget 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

--2025-12-05 13:38:08--  https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg
Resolving habrastorage.org (habrastorage.org)... 95.47.173.35, 95.47.173.34, 2a14:b680:0:56::34, ...
Connecting to habrastorage.org (habrastorage.org)|95.47.173.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398272 (389K) [image/jpeg]
Saving to: ‘yf_dokzqy3vcritme8ggnzqlvwa.jpeg’

yf_dokzqy3vcritme8g 100%[===================>] 388.94K  1.07MB/s    in 0.4s    

2025-12-05 13:38:09 (1.07 MB/s) - ‘yf_dokzqy3vcritme8ggnzqlvwa.jpeg’ saved [398272/398272]



In [15]:
img = prepare_image(download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'),(200,200))

In [10]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 1) Conv layer: in 3 ch, out 32 ch, kernel 3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        # 2) Max pooling: 2x2
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        # After conv1 (no padding): input (3,200,200) -> (32, 198, 198)
        # After maxpool 2x2: -> (32, 99, 99)
        # Flatten size = 32 * 99 * 99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()  # binary output

    def forward(self, x):
        # x: (batch, 3, 200, 200)
        x = self.conv1(x)          # (batch, 32, 198, 198)
        x = F.relu(x)
        x = self.pool(x)           # (batch, 32, 99, 99)
        x = torch.flatten(x, 1)    # (batch, 32*99*99)
        x = self.fc1(x)            # (batch, 64)
        x = F.relu(x)
        x = self.fc2(x)            # (batch, 1)
        x = self.sigmoid(x)        # probability for class 1
        return x

In [12]:
model = SimpleCNN()

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

criterion = nn.MSELoss()

In [13]:
from PIL import Image
import torch
from torchvision import transforms

# 1) Define same size, channels-first, and normalization if you want
transform = transforms.Compose([
    transforms.Resize((200, 200)),     # model expects 200x200 [web:35]
    transforms.ToTensor(),             # -> (C, H, W), values in [0, 1] [web:36]
    # Optional: add Normalize(...) here if you trained with it [web:39]
])

# 2) Load and transform image
img = img.convert("RGB")  # 3 channels [web:37]
x = transform(img)            # shape (3, 200, 200)
x = x.unsqueeze(0)            # shape (1, 3, 200, 200) add batch dim [web:35]

In [14]:
model = SimpleCNN()
model.eval()                                                            # eval mode [web:37]

with torch.no_grad():
    y = model(x)              # shape (1, 1)
    prob = y.item()           # scalar probability for class 1

# For binary decision:
pred = 1 if prob >= 0.5 else 0
print("prob:", prob, "pred:", pred)

prob: 0.4977841079235077 pred: 0


In [15]:
%%shell
pip install udocker
udocker --allow-root install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 9.6 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.17
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful


In [2]:
import requests

repo = "agrigorev/model-2025-hairstyle"
tag = "v1"

# 获取 token
auth_url = f"https://auth.docker.io/token?service=registry.docker.io&scope=repository:{repo}:pull"
token = requests.get(auth_url).json()["token"]

# 获取 manifest
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.docker.distribution.manifest.v2+json"
}
manifest_url = f"https://registry-1.docker.io/v2/{repo}/manifests/{tag}"
manifest = requests.get(manifest_url, headers=headers).json()

# 输出 digest
digest = manifest["config"]["digest"]
print("Manifest digest:", digest)

# 计算镜像总大小
total_size = sum(layer["size"] for layer in manifest["layers"])
print("Total size (bytes):", total_size)

Manifest digest: sha256:4528ad1525d55b7b0c0872d6b7882b654162fcc05a013e455719ac0c0eb44ad3
Total size (bytes): 268965283


In [3]:
for layer in manifest["layers"]:
    layer_digest = layer["digest"]
    url = f"https://registry-1.docker.io/v2/{repo}/blobs/{layer_digest}"
    headers = {"Authorization": f"Bearer {token}"}
    r = requests.get(url, headers=headers, stream=True)
    layer_file = layer_digest.replace(":", "_") + ".tar.gz"
    with open(layer_file, "wb") as f:
        for chunk in r.iter_content(1024*1024):
            f.write(chunk)
    print(f"Saved {layer_file}")

Saved sha256_b71754c34aa30fa068984c76f3eaf68b76ee2e54328b98dcf2b30a00786c2fd0.tar.gz
Saved sha256_d2379533db7f1f8d28362676e66255b190981a48c09f71c5349d1c430b486da2.tar.gz
Saved sha256_26c6d8a1e1c2efebb092f81375307e44c7a24372b57f330bbc8cf04aa0673ec4.tar.gz
Saved sha256_94f2ecca3b37ec53066991d968d105de9ec006f858ca315b4349d27072eea86e.tar.gz
Saved sha256_f9e394d707b7723a883f06ec18a667e9dfb92afb8adee558b6002a0baed96df1.tar.gz
Saved sha256_15b946295de28858d1ff78561548e1daa544168847ff31cef0a10e2b18a50ff5.tar.gz
Saved sha256_55d9a27bb275e3e351c7135a444865ff4267198ee09af922f6776a8a42633abc.tar.gz
Saved sha256_cd9c39ee4ab89af1dee8507f2bde9f1c1e503cb515fcd91daebbb8ac52b910c6.tar.gz


In [6]:
import tarfile
import os

layer_files = [
    "/content/sha256_15b946295de28858d1ff78561548e1daa544168847ff31cef0a10e2b18a50ff5.tar.gz",
    "/content/sha256_26c6d8a1e1c2efebb092f81375307e44c7a24372b57f330bbc8cf04aa0673ec4.tar.gz",
    "/content/sha256_55d9a27bb275e3e351c7135a444865ff4267198ee09af922f6776a8a42633abc.tar.gz",
    "/content/sha256_94f2ecca3b37ec53066991d968d105de9ec006f858ca315b4349d27072eea86e.tar.gz",
    "/content/sha256_b71754c34aa30fa068984c76f3eaf68b76ee2e54328b98dcf2b30a00786c2fd0.tar.gz",
    "/content/sha256_cd9c39ee4ab89af1dee8507f2bde9f1c1e503cb515fcd91daebbb8ac52b910c6.tar.gz",
    "/content/sha256_d2379533db7f1f8d28362676e66255b190981a48c09f71c5349d1c430b486da2.tar.gz",
    "/content/sha256_f9e394d707b7723a883f06ec18a667e9dfb92afb8adee558b6002a0baed96df1.tar.gz",
]

# 解压到同一个文件夹，后面的 layer 会覆盖前面的
os.makedirs("/content/full_image", exist_ok=True)

for layer_file in layer_files:
    print(f"Extracting {layer_file}...")
    with tarfile.open(layer_file, "r:") as tar:
        tar.extractall(path="/content/full_image")

Extracting /content/sha256_15b946295de28858d1ff78561548e1daa544168847ff31cef0a10e2b18a50ff5.tar.gz...


ReadError: truncated header